In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
formula = """
    ClaimCount ~ 
    TE(Region) + 
    BonusMalus + 
    I(BonusMalus ** 2) + 
    ms(VehAge, df=4) + 
    pos(DrivAge) + 
    pos(I(DrivAge ** 2)) + 
    TE(VehBrand) + 
    TE(Area), 
"""

result = rs.glm(
    formula,
    train_data, 
    family="poisson", 
    offset="Exposure").fit(
)

print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406427
Method:              IRLS            Df Model:                    11
Scale:               1.0000          Iterations:                  25

Log-Likelihood:         -118547.9586 Deviance:                195548.2674
AIC:                     237119.9171 Null Deviance:           132328.3090
BIC:                     237250.8994 Pearson chi2:              390504.96
Converged:           False          

------------------------------------------------------------------------------
Variable                   Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept               -4.2190     0.0721  -58.518  <0.0001   [ -4.3603,  -4.0777]  ***
BonusMalus               0.0319     0.0013   25

In [4]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': '\n    ClaimCount ~ \n    TE(Region) + \n    BonusMalus + \n    I(BonusMalus ** 2) + \n    ms(VehAge, df=4) + \n    pos(DrivAge) + \n    pos(I(DrivAge ** 2)) + \n    TE(VehBrand) + \n    TE(Area), \n', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 12, 'df_resid': 406427, 'converged': False, 'iterations': 25}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=84258.07, loss=0.504328, deviance=204978.7, log_likelihood=-123263.17, aic=246550.34, gini=0.1575, auc=0.5788, ae_ratio=0.2541, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1715.74, actual=0.609649, predicted=0.262003, ae_ratio=2.3269), DecileMetrics(decile=2, n=40643, exposure=3747.26, actual=0.304222, predicted=0.29789, ae_ratio=1.0213), DecileMetrics(decile=3, n=40643, exposure=7596.25, actual=0.181669, predicted=0.28573, ae_ratio=0.6358), DecileMetrics(decil